(hhl_algorithm)=

# Алгоритм HHL

Мы с вами растём и умнеем не по дням, а по часам, – с настоящим квантовым ускорением – и сегодня пришла пора поговорить о знаменитом алгоритме Харроу, Хиссадима и Ллойда, более известном как HHL-алгоритме, способном решать системы линейных уравнений.

Очень надеюсь, что к данному занятию у вас уже есть представление об алгоритме фазовой оценки (QPE), использующем обратное квантовое преобразование Фурье, на котором и базируется HHL. Глубокое понимание всех тонкостей этого алгоритма потребует от вас уверенного владения математическим аппаратом. За детальным описанием вы всегда можете обратиться к статьям ["Quantum linear systems algorithms: a primer"](https://arxiv.org/abs/1802.08227), ["Quantum algorithm for solving linear systems of equations"](https://arxiv.org/abs/0811.3171v3), и ["Homomorphic Encryption Experiments on IBM's Cloud Quantum Computing Platform"](https://arxiv.org/abs/1612.02886v2). Приготовьтесь потратить время и умственные ресурсы, если алгоритм вас зацепит и вы решите в нём как следует покопаться. Мы же поможем вам заинтересоваться, рассмотрим основные принципы и небольшой пример.

Представим обычную систему линейных уравнений:

$$
\left\{\begin{array}{l}
a_{11} x_{1}+a_{12} x_{2} = b_{1} \\
a_{21} x_{1}+a_{22} x_{2} = b_{2}
\end{array}\right.
$$

или иначе:

$$Ax = b, A|x\rangle = |b\rangle$$

Чтобы найти искомый вектор $X$, всё что нам по сути нужно сделать -- это найти оператор $А$ в минус первой степени и применить его к вектору $b$:

$$x = A^{-1}b$$

Оказывается, что и это всё можно провернуть с помощью известных квантовых преобразований. Принципиальный вид нашей схемы представлен следующим образом:

```{figure} /_static/qcblock/hhl_algorithm/hhl_curcuit.png
:name: hhl_curcuit
:width: 800px

Квантовая схема, реализующая алгоритм HHL
```

В нижний регистр загружается вектор $b$, средний и нижний регистры участвуют в фазовой оценке, а верхний дополнительный кубит нужен для так называемого вращения, обусловленного собственными значениями. Давайте разбираться.

Для начала мы должны подготовить наши регистры по всем квантовым законам: $b$ и $x$ должны быть пронормированы, а оператор $A$ должен быть эрмитовым (напомню, что эрмитов оператор при транспонировании равен комплексно сопряжённому – найдите, перечитайте и ещё раз осмыслите этот факт, он нам нужен). Это важно с точки зрения ортогональности собственных векторов (вспомните про ортонормированный базис, сферу Блоха, комплексное представление векторов $|0\rangle$ и $|1\rangle$ и пр.), а также унитарности, иначе говоря, сохранения энергии: всё, что мы проделываем с двумя нижними регистрами, должно быть обратимо; если мы изменили кубит с помощью оператора – мы должны легко вернуть его в первоначальное состояние этим же оператором, т.е. не меняя его энергию.

Мы будем использовать оператор $U = e^{iAt}$, и нужно, чтобы он был обратим – для этого $А$ должна быть эрмитовой.

Если матрица $А$ изначально не является таковой, существует способ искусственно привести её к нужному виду через тензорное произведение и дополнительные кубиты для второго регистра, векторов $b$ и $x$. [можно вставить картинку с общим видом матрицы] Мы рассмотрим более простой пример.

Но прежде мы должны воспользоваться одним важным свойством: эрмитову матрицу $А$ представим в виде суммы собственных векторов, умноженных на собственные значения, т.е. сделаем спектральное разложение:

$$
\begin{aligned}
&A=\sum_{j=0}^{N-1} \lambda_{j}\left|u_{j}\right\rangle\left\langle u_{j}\right| \\
&A^{-1}=\sum_{j=0}^{N-1} \lambda_{j}^{-1}\left|u_{j}\right\rangle\left\langle u_{j}\right|
\end{aligned}
$$

Тогда векторы $b$ и $x$ мы можем также представить через собственные вектора $A$:

$$
|b\rangle = \sum_{j=0}^{N-1} b_{j} | u_j \rangle, \ Ax = \lambda x
$$

Чтобы понять, почему это тоже ключевой момент, давайте вспомним, что значит собственный вектор и собственное значение матрицы.

Представьте океан с хаотичным движением кучи маленьких волн. Каждую такую волну можно рассматривать как вектор, у которого есть направление и скорость – т.е. его длина (обозначенные зелёным цветом).


```{figure} /_static/qcblock/hhl_algorithm/ocean_and_vectors.png
:name: ocean_and_vectors
:width: 600px

Иллюстрация собственных векторов и значений
```

Представим также, что существует некоторое упорядоченное течение в виде оператора $A$. Все векторы в нём (жёлтый цвет) имеют определённое направление, но могут быть разными по длине, то есть скорости. Это течение воздействует на все попадающиеся ему волны в океане, и большинство волн после его воздействия искажаются и меняют своё направление. Но если случайная волна (например, обозначенная фиолетовым цветом) имеет то же направление, что и течение $А$, то такая волна не изменит курс. Её-то и можно абстрактно назначить собственным вектором $х$ оператора $А$ с собственным значением лямбда:

$$Ax = \lambda x$$

Таким образом, искомый вектор $х$ -- не что иное как:

$$|x\rangle = A^{-1} |b \rangle = \sum_{j=0}^{N-1} \lambda_{j}^{-1}b_j | u_{j}\rangle
$$


Итак, фазовая оценка. Мы применяем к кубитам второго регистра матрицы Адамара, тем самым приводим их в суперпозицию. Следом запускаем оператор $U$:

$$
U = e^{iAt} = \sum_{j=0}^{N-1} e^{i \lambda_j t} \left| u_j \rangle \langle u_j \right|
$$

Наверняка многие увидят сходство формулы с уравнением Шрёдингера, описывающем эволюцию системы с течением времени (которую, кстати, тоже можно «разложить» на собственные векторы и собственные значения), и аналогом гамильтониана здесь выступает матрица $А$.

U «находит» собственные значения $А$ и записывает их в виде фазы в том случае, если кубит второго регистра находится в состоянии $|1\rangle$:


```{figure} /_static/qcblock/hhl_algorithm/hhl_curcuit2.png
:name: hhl_curcuit2
:width: 800px
```

Таким образом, состояние второго регистра приходит в вид:

$$\large |0\rangle + e^{2\pi i 2^{m-1} \psi} |1\rangle + |0\rangle + e^{2\pi i 2^{m-2} \psi} |1\rangle + \dots +  |0\rangle + e^{2\pi i 2^{0} \psi} |1\rangle = \sum_{l=0}^{2^{m-1}} e^{2\pi i \psi l} |l\rangle
$$

Однако просто записать фазу недостаточно. При измерении мы можем получить единицу, но как узнать её вероятность? Измерение не даст нам этой цифры, а ноль и вовсе неинформативен.
Алгоритм обратного квантового Фурье переводит фазу в конкретный вектор.

Так что на выходе после QPE мы имеем:

```{figure} /_static/qcblock/hhl_algorithm/qft_output.png
:name: qft_output
:width: 600px

```

Дело осталось за малым. Нам нужно всего-то перевернуть все лямбды и вернуть им вероятностную, а не векторную форму. За это отвечает специальный оператор вращения $R$:

$$
R|0\rangle = \sum_{i=0}^{N-1}\left(\sqrt{1-\frac{c^{2}}{\lambda_{j}^{2}}}|0\rangle + \frac{c}{\lambda_{j}}|1\rangle\right),
$$

где $С$ -- константа, которая должна быть меньше минимального из лямбда: $|C| < \lambda_{min}$ [Почему?].

После вращения и обратного QPE состояния будут следующими:

\begin{aligned}
&\sum_{j=0}^{N-1}\left(\sqrt{1-\frac{c^{2}}{\lambda_{j}^{2}}}|0\rangle+\frac{c}{\lambda_j}|1\rangle\right) b_j\left|\lambda_{j}\right\rangle_{n}\left|u_{j}\right\rangle_{m} \\
&\sum_{j=0}^{N-1}\left(\sqrt{1-\frac{c^{r}}{\lambda_{i}^{2}}}|0\rangle+\frac{c}{\lambda_j}|1\rangle\right) b_{j}|0\rangle_{n}\left|u_{j}\right\rangle_{m}
\end{aligned}

В конце мы измеряем верхний кубит и если получаем единицу, то знаем, что в нижнем регистре хранится искомый $х$ с учётом нормировки:


```{figure} /_static/qcblock/hhl_algorithm/hhl_curcuit3.png
:name: hhl_curcuit3
:width: 800px
```

## Пример

Рассмотрим небольшой, но удобный пример. Удобный в том отношении, что, вообще говоря, алгоритм HHL имеет определённое приближение. Если собственные значения не представимы в бинарной форме, то о 100% точности говорить не приходится. Мы также опустим ряд вопросов, связанных с подбором параметров и количества кубитов второго регистра. Главное сейчас -- понять, что происходит, и для этого наша матрица $А$ эрмитова, все условия подобраны, а преобразования точны. Стоит помнить, что знать заранее значение собственных векторов и собственных значений нам совершенно не обязательно -- это нужно лишь для наглядности.

Итак, пусть мы имеем условие:

$$
\large
\begin{aligned}
&A=\left(\begin{array}{ll}
1 & \frac{3}{5} \\
\frac{3}{5} & 1
\end{array}\right) \\
&|b\rangle=|1\rangle=\left[\begin{array}{l}
0 \\
1
\end{array}\right]
\end{aligned}
$$

$$
\large
\left\{\begin{array}{l}
x_{1}+\frac{3}{5} x_{2}=0 \\
\frac{3}{5} x_{1}+x_{2}=1
\end{array}\right.
$$

$$
\large
\begin{aligned}
&\lambda_{0}=\frac{2}{5},\left|u_{0}\right\rangle=\left[\begin{array}{c}
-1 \\
1
\end{array}\right] \\
&\lambda_{1}=\frac{8}{5},\left|u_{1}\right\rangle=\left[\begin{array}{l}
1 \\
1
\end{array}\right]
\end{aligned}
$$

Вектор $b$ мы легко можем представить через собственные вектора $u_j$:


```{figure} /_static/qcblock/hhl_algorithm/b_u_j_composition.png
:name: b_u_j_composition
:width: 400px
```

Зададим параметр $t$ и проанализируем фазу:

$$\large
\mathord{\sqsupset} \ t = 2\pi \frac{5}{16}
$$

$$\large
e^{2\pi i \psi} = e^{i \lambda_j t}, \psi = \frac{\lambda_j t}{2\pi}$$

```{figure} /_static/qcblock/hhl_algorithm/hhl_example.png
:name: hhl_example.png
:width: 400px
```

Как мы видим, для перевода угла $\psi$ в векторную форму, нам понадобятся три кубита. После преобразования QPE мы имеем следующее состояние:

$$
\begin{aligned}
&{\left[|b\rangle_{m}=\sum_{j=0}^{1} \frac{1}{\sqrt{2}}\left|u_{j}\right\rangle\right]} \\
&\frac{1}{\sqrt{2}}\left(|001\rangle\left|u_{0}\right\rangle+|100\rangle\left|u_{1}\right\rangle\right)
\end{aligned}
$$

Подберём константу и произведём вращение:

$$\large
c =  \frac{1}{16}

\begin{aligned}
&\frac{1}{\sqrt{2}}|001\rangle\left|u_{0}\right\rangle\left(\sqrt{1-\frac{(1 / 16)^{2}}{(1 / 8)^{2}}}|0\rangle+\frac{1 / 16}{1 / 8}|1\rangle\right) +\\
&+\frac{1}{\sqrt{2}}|100\rangle\left(u_{1}\right)\left(\sqrt{1-\frac{(1 / 16)^{2}}{(1 / 2)^{2}}}|0\rangle+\frac{1 / 16}{1 / 2}|1\rangle\right) \\
&\frac{1}{\sqrt{2}}|000\rangle\left|u_{0}\right\rangle \frac{1}{2}|1\rangle+\frac{1}{\sqrt{2}}|000\rangle\left(u_{1}\right\rangle \frac{1}{8}(1)
\end{aligned}

\Rightarrow \sqrt{\frac{17}{128}}
$$



В конце мы производим измерение верхнего кубита и при получении единицы можем быть уверены, что нижний регистр содержит искомое решение с учётом нормировки.